In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
dataset=pd.read_csv('CKD.csv')
dataset=pd.get_dummies(dataset,drop_first=True,dtype='int')
dataset

age         bp   al   su         bgr          bu        sc  \
0     2.000000  76.459948  3.0  0.0  148.112676   57.482105  3.077356   
1     3.000000  76.459948  2.0  0.0  148.112676   22.000000  0.700000   
2     4.000000  76.459948  1.0  0.0   99.000000   23.000000  0.600000   
3     5.000000  76.459948  1.0  0.0  148.112676   16.000000  0.700000   
4     5.000000  50.000000  0.0  0.0  148.112676   25.000000  0.600000   
..         ...        ...  ...  ...         ...         ...       ...   
394  51.492308  70.000000  0.0  0.0  219.000000   36.000000  1.300000   
395  51.492308  70.000000  0.0  2.0  220.000000   68.000000  2.800000   
396  51.492308  70.000000  3.0  0.0  110.000000  115.000000  6.000000   
397  51.492308  90.000000  0.0  0.0  207.000000   80.000000  6.800000   
398  51.492308  80.000000  0.0  0.0  100.000000   49.000000  1.000000   

            sod       pot       hrmo  ...  pc_normal  pcc_present  ba_present  \
0    137.528754  4.627244  12.518156  ...          0            0           0   
1    137.528754  4.627244  10.700000  ...          1            0           0   
2    138.000000  4.400000  12.000000  ...          1            0           0   
3    138.000000  3.200000   8.100000  ...          1            0           0   
4    137.528754  4.627244  11.800000  ...          1            0           0   
..          ...       ...        ...  ...        ...          ...         ...   
394  139.000000  3.700000  12.500000  ...          1            0           0   
395  137.528754  4.627244   8.700000  ...          1            0           0   
396  134.000000  2.700000   9.100000  ...          1            0           0   
397  142.000000  5.500000   8.500000  ...          1            0           0   
398  140.000000  5.000000  16.300000  ...          1            0           0   

     htn_yes  dm_yes  cad_yes  appet_yes  pe_yes  ane_yes  classification_yes  
0          0       0        0          1       1        0                   1  
1          0       0        0          1       0        0                   1  
2          0       0        0          1       0        0                   1  
3          0       0        0          1       0        1                   1  
4          0       0        0          1       0        0                   1  
..       ...     ...      ...        ...     ...      ...                 ...  
394        0       0        0          1       0        0                   1  
395        1       1        0          1       0        1                   1  
396        1       1        0          0       0        0                   1  
397        1       1        0          1       0        1                   1  
398        0       0        0          1       0        0                   0  

[399 rows x 28 columns]

In [3]:
dataset.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [4]:
dataset['classification_yes'].value_counts()

classification_yes
1    249
0    150
Name: count, dtype: int64

In [5]:
indep=dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]
dep=dataset[['classification_yes']]

In [10]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(indep,dep,test_size=0.30,random_state=0)

In [11]:
from sklearn.naive_bayes import ComplementNB
from sklearn.model_selection import GridSearchCV
param_grid={'alpha':[0.001,0.01,1.0],'force_alpha':[True,'bool'],'fit_prior':[True,'bool'],'norm':[False,'bool']}
grid=GridSearchCV(ComplementNB(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(x_train,y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


C:\Anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
105 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Anaconda3\Lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "C:\Anaconda3\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "C:\Anaconda3\Lib\site-packages\sklearn\utils\_param_validation.py", line 95, in validate_

GridSearchCV(estimator=ComplementNB(), n_jobs=-1,
             param_grid={'alpha': [0.001, 0.01, 1.0],
                         'fit_prior': [True, 'bool'],
                         'force_alpha': [True, 'bool'],
                         'norm': [False, 'bool']},
             scoring='f1_weighted', verbose=3)

In [12]:
result=grid.cv_results_
grid_prediction=grid.predict(x_test)

In [13]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,grid_prediction)
print(cm)

[[44  1]
 [12 63]]


In [14]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,grid_prediction)
print(clf_report)

              precision    recall  f1-score   support

           0       0.79      0.98      0.87        45
           1       0.98      0.84      0.91        75

    accuracy                           0.89       120
   macro avg       0.89      0.91      0.89       120
weighted avg       0.91      0.89      0.89       120



In [15]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_prediction,average='weighted')
print('The f1_macro value for the best parameters {}'.format(grid.best_params_),f1_macro)

The f1_macro value for the best parameters {'alpha': 0.001, 'fit_prior': True, 'force_alpha': True, 'norm': False} 0.8932794358572548


In [16]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])

0.9525925925925927

In [17]:
table=pd.DataFrame.from_dict(result)
table

mean_fit_time  std_fit_time  mean_score_time  std_score_time param_alpha  \
0        0.220180      0.153854         0.036841        0.048257       0.001   
1        0.001548      0.001899         0.000000        0.000000       0.001   
2        0.001004      0.002008         0.000000        0.000000       0.001   
3        0.026364      0.041031         0.000000        0.000000       0.001   
4        0.006272      0.007681         0.000000        0.000000       0.001   
5        0.035696      0.055678         0.000000        0.000000       0.001   
6        0.003243      0.004065         0.000000        0.000000       0.001   
7        0.016812      0.018471         0.000000        0.000000       0.001   
8        0.170151      0.076352         0.038195        0.015399        0.01   
9        0.004050      0.002169         0.000000        0.000000        0.01   
10       0.006270      0.005953         0.000000        0.000000        0.01   
11       0.012288      0.012651         0.000000        0.000000        0.01   
12       0.025832      0.034913         0.000000        0.000000        0.01   
13       0.002810      0.005621         0.000000        0.000000        0.01   
14       0.001004      0.002008         0.000000        0.000000        0.01   
15       0.009831      0.010199         0.000000        0.000000        0.01   
16       0.053162      0.044423         0.037748        0.030312         1.0   
17       0.002009      0.002720         0.000000        0.000000         1.0   
18       0.000000      0.000000         0.000000        0.000000         1.0   
19       0.004809      0.003870         0.000000        0.000000         1.0   
20       0.011458      0.016389         0.000000        0.000000         1.0   
21       0.003449      0.003345         0.000000        0.000000         1.0   
22       0.002816      0.003124         0.000000        0.000000         1.0   
23       0.004309      0.004640         0.000000        0.000000         1.0   

   param_fit_prior param_force_alpha param_norm  \
0             True              True      False   
1             True              True       bool   
2             True              bool      False   
3             True              bool       bool   
4             bool              True      False   
5             bool              True       bool   
6             bool              bool      False   
7             bool              bool       bool   
8             True              True      False   
9             True              True       bool   
10            True              bool      False   
11            True              bool       bool   
12            bool              True      False   
13            bool              True       bool   
14            bool              bool      False   
15            bool              bool       bool   
16            True              True      False   
17            True              True       bool   
18            True              bool      False   
19            True              bool       bool   
20            bool              True      False   
21            bool              True       bool   
22            bool              bool      False   
23            bool              bool       bool   

                                               params  split0_test_score  \
0   {'alpha': 0.001, 'fit_prior': True, 'force_alp...           0.929513   
1   {'alpha': 0.001, 'fit_prior': True, 'force_alp...                NaN   
2   {'alpha': 0.001, 'fit_prior': True, 'force_alp...                NaN   
3   {'alpha': 0.001, 'fit_prior': True, 'force_alp...                NaN   
4   {'alpha': 0.001, 'fit_prior': 'bool', 'force_a...                NaN   
5   {'alpha': 0.001, 'fit_prior': 'bool', 'force_a...                NaN   
6   {'alpha': 0.001, 'fit_prior': 'bool', 'force_a...                NaN   
7   {'alpha': 0.001, 'fit_prior': 'bool', 'force_a...                NaN   
8   {'alpha': 0.01, 'fit_prior': True, 'for